In [323]:
import numpy as np

"""
This function returns false for the 0 string.
"""

def false_s(s):
    return s != '0'

"""
This function returns a list of modes in x.
"""

def mode_fn(x):
    return [i for i in set(x) if x.count(i) == max(map(x.count, x))]

"""
This function recursively looks for a valid value in protein
strand s at index i in the priority queue arr. This function
returns an invalid value if a valid one is not available.
"""

def r_check(arr, s, i):
    if false_s(arr[s][i]):
        return arr[s][i]
    
    next = s + 1
    
    if next == len(arr):
        return '0'
    
    r_check(arr, next, i)
    

"""
This function returns the most common amino acid among the
proteins in arr with ancestral priority of list p at index i.
"""

def checker(arr, p, i):
    r = [s[i] for s in arr]
    m = mode_fn(r)
    
    if len(m) == 1:
        return m[0] if false_s(m[0]) else r_check(p, 0, i)
    
    lst = [c for c in p if c[i] in m]
    
    return r_check(lst, 0, i)

    
"""
This function predicts the ancestral protein of the proteins in arr
that are all of the same length. Use fillers to accommodate lengths.
"""

def predict(arr, p):
    m = max(map(len, arr))
    pro = [checker(arr, p, i) for i in range(m)]
    return str().join(pro)

"""
This function prints out the locations in s that are NA.
"""

def find_na(s):
    for i, c in enumerate(s):
        if c == '0':
            print(i)
           
"""
This function returns the shorter strand.
"""
            
def shorter(a, b):
    return a if len(a) < len(b) else b

"""
This function assists in the comparison between strands of proteins.
"""

def compare(a, b):
    l = len(shorter(a, b))
    c = [a[i] == b[i] for i in range(l)]
    return sum(c)

"""
This function provides a list of the number of similarities between
s and each of the strands in arr.
"""

def counter(arr, s):
    return [compare(pro, s) for pro in arr]

"""
This function assists with recursively inserting placeholders into
s at the locations in lst.
"""

def r_fills(s, lst):
    if lst:
        return r_fills(s[:lst[0]] + '0' + s[lst[0]:], lst[1:])
    else:
        return s
    
"""
This function simulates the insertion of n placeholders to compare 
the similar proteins between s and strands in arr. This simulation
returns the indices of where n placeholders should be for the
maximum number of similarities.
"""
    
def fillers(arr, s, n):
    empty = [len(s) + 1 for i in range(n)]
    value = np.zeros(empty)
    it = np.nditer(value, flags = ['multi_index'], op_flags = ['readwrite'])
    
    while not it.finished:
        it[0] = sum(counter(arr, r_fills(s, it.multi_index)))
        it.iternext()

    return np.unravel_index(value.argmax(), value.shape)

In [324]:
# Here is an example with the protein strand for epidermal growth factor in transmembrane domain!

In [325]:
# protein strand of length 24 for zebrafish
z = 'MVIAVCIVLLITILSIAACITFCY'

# protein strand of length 24 for frog
f = 'VTIAVSLLLLLLILGLGSFATYYY'

In [326]:
# protein strand of length 23 for human
h = 'IATGMVGALLLLLVVALGIGLFM'

# protein strand of length 22 for chicken
c = 'ITIAVCIAVLLLLLGSLAAYCS'

# protein strand fo length 21 for dog
d = 'VAAVAVGVVVLVLLLLLGLGG' 

In [327]:
"""
Let us use the fillers function to provide placeholders for human, chicken, and dog based on
the strands of zebrafish and dog. We start this by including zebrafish and dog in a list.
"""
lst = [z, f]
lst

['MVIAVCIVLLITILSIAACITFCY', 'VTIAVSLLLLLLILGLGSFATYYY']

In [328]:
"""
Now we can use fillers function to provide placeholders for human where 1 is the difference
between the length of the human strand and the zebrafish or frog strand.
"""
h_i = fillers(lst, h, 1)

"""
12 is the location where 1 placeholder would make the human strand have the most amino acids
in common with those of the zebrach and frog strands.
"""
h_i

(12,)

In [329]:
"""
Let us use r_fills to fill the human strand with a placeholder at this position.
"""
h_n = r_fills(h, h_i)

"""
Now we can see that plceholder at position 12.
"""
h_n

'IATGMVGALLLL0LVVALGIGLFM'

In [330]:
"""
Now we can include this new simulation of a human strand in the list of strands that have 
the same length.
"""
lst.append(h_n)


"""
Here is that new list.
"""
lst

['MVIAVCIVLLITILSIAACITFCY',
 'VTIAVSLLLLLLILGLGSFATYYY',
 'IATGMVGALLLL0LVVALGIGLFM']

In [331]:
# Let us repeat the above steps for chicken, which has 2 fewer amino acids for that protein.
c_i = fillers(lst, c, 2)

# Fill with placeholders.
c_n = r_fills(c, c_i)

# Add this new strand to the list.
lst.append(c_n)

# Let us repeat the above steps for dog, which has 3 fewer amino acids for that protein.
d_i = fillers(lst, c, 3)

# Fill with placeholders.
d_n = r_fills(d, d_i)

# Add this new strand to the list.
lst.append(d_n)

# Let us view our new list with all strands at a length of 24.
lst

['MVIAVCIVLLITILSIAACITFCY',
 'VTIAVSLLLLLLILGLGSFATYYY',
 'IATGMVGALLLL0LVVALGIGLFM',
 'ITIAVCIAVLLL0L0LGSLAAYCS',
 'VAAVAVGVVVLV0L0LLLLGLGG0']

In [333]:
"""
Now we can attempt to predict the ancestral strand of this protein by finding the mode of overlap 
for each location. If there is none, we take the amino acid at that location of the strand based 
on a priority queue, which places the strands of the older species first. Let us create that 
priority queue.
"""
queue = [z, f, c, d, h]

"""
Now we can run the predict function to see what the ancestral strand may have been.
"""
a = predict(lst, queue)

In [334]:
# Here is the ancestral strand prediction!
a

'VTIAVCIVLLLLILSLASLITYCY'